In [26]:

from components.controller.bluetooth_controller import BlueToothCarControlSPP
from components.controller.bluetooth_SPP_server import start_bluetooth_server_v2
from components.gyroscope.gyroscope import AngularSpeedControlV2
from components.two_wheels import TwoWheelsV2
from components.camera import PicameraV2
from components.controller.image_ml import ImageMLController
from data_collection.data_collection import LoggerSet, Logger
from multiprocessing import Manager
from components import get_switches

from components import default_loop_v2, default_component_process_starter_v2, shared_value
import datetime
import RPi.GPIO as GPIO
GPIO.setmode(GPIO.BOARD)



%load_ext autoreload
%autoreload 2

In [18]:
logger_set = LoggerSet('../log/session'+str(datetime.datetime.now()), overwrite_ok=False)
manager = Manager()

## Two Wheels

In [19]:
w_out, w_starter = default_component_process_starter_v2(
    TwoWheelsV2, 
    init_kwargs= dict(
        left_pin=33, 
        right_pin=32, 
        dir_pins=(16, 18), 
        logger_set=logger_set, 
        name='TwoWheelsV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
imu_out, imu_starter = default_component_process_starter_v2(
    AngularSpeedControlV2, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='AngularSpeedControlV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
)
cam_out, cam_starter = default_component_process_starter_v2(
    PicameraV2, 
    init_kwargs= dict(
        resolution=(114, 64), framerate=30,
        logger_set=logger_set, 
        name='PicameraV2'
    ), 
    mainloop = default_loop_v2, 
    main_kwargs=dict(interval=1/50),
    manager=manager, 
    shared_outputs_kwargs=dict(resolution=(114, 64))
)

bt_ser_out, bt_ser = start_bluetooth_server_v2(manager)

bt_out, bt_starter = default_component_process_starter_v2(
    BlueToothCarControlSPP, 
    init_kwargs= dict(
        logger_set=logger_set, 
        name='BlueToothCarControlSPP'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/30), 
    manager=manager, 
)


w = w_starter(imu_out)
imu = imu_starter(bt_out+[lambda: bt_ser_out[0]().get('pause')])
cam = cam_starter()
bt = bt_starter(bt_ser_out)

Starting Serial Port Profile...


/home/kawa/projects/car1/src/components/two_wheels.py:15: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(pin, GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:73: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[0], GPIO.OUT) #type: ignore
/home/kawa/projects/car1/src/components/two_wheels.py:74: RuntimeWarning: This channel is already in use, continuing anyway.  Use GPIO.setwarnings(False) to disable warnings.
  GPIO.setup(dir_pins[1], GPIO.OUT) #type: ignore
[2:37:09.179603034] [16674]  INFO Camera camera_manager.cpp:297 libcamera v0.0.5+83-bde9b04f


[2:37:09.234283104] [16688]  INFO RPI vc4.cpp:437 Registered camera /base/soc/i2c0mux/i2c@1/ov5647@36 to Unicam device /dev/media1 and ISP device /dev/media2
[2:37:09.234415621] [16688]  INFO RPI pipeline_base.cpp:1101 Using configuration file '/usr/share/libcamera/pipeline/rpi/vc4/rpi_apps.yaml'
[2:37:09.255741806] [16674]  INFO Camera camera.cpp:1033 configuring streams: (0) 114x64-RGB888 (1) 1920x1080-SGRBG10_CSI2P
[2:37:09.256384059] [16688]  INFO RPI vc4.cpp:565 Sensor: /base/soc/i2c0mux/i2c@1/ov5647@36 - Selected sensor format: 1920x1080-SGRBG10_1X10 - Selected unicam format: 1920x1080-pgAA


NewConnection(/org/bluez/hci0/dev_C8_BD_4D_BA_4F_15, 101)
reset
reset
reset
reset
reset
reset
reset
reset
reset
saving: TwoWheelsV2
saving: AngularSpeedControlV2
saving: PicameraV2
saving: BlueToothCarControlSPP
saving: TwoWheelsV2
saving: PicameraV2saving: BlueToothCarControlSPP

saving: AngularSpeedControlV2
saving: TwoWheelsV2
saving: BlueToothCarControlSPP
saving: PicameraV2
saving: AngularSpeedControlV2


In [8]:
bt_out[0]()

0

In [10]:
x = lambda: bt_ser_out[0]().get('pause')
x()

True

In [17]:
w.terminate()
imu.terminate()
cam.terminate()
bt_ser.terminate()
bt.terminate()


# ML Component


In [186]:
ml_out, ml_starter = default_component_process_starter_v2(
    ImageMLController, 
    init_kwargs= dict(
        #resolution=(114, 64), framerate=30,
        model_path='test.keras',
        logger_set=logger_set, 
        name='ImageMLController'
    ), 
    mainloop=default_loop_v2, 
    main_kwargs=dict(interval=1/60), 
    manager=manager, 
)
ml = ml_starter(cam_out)

In [194]:
ml.terminate()

In [3]:
d = {'a':1, 'b':False}

if b:= d.get('b'):
    print (b)